In [2]:
import os
import numpy as np
import tensorflow as tf


def load_minibatch(index, var):
    shape = np.load(os.path.join('DATA', 'MINIBATCHES', f'{index}_shape.npy'))
    for i in range(len(shape)):
        if var.shape[i] == None:
            continue
        elif var.shape[i] != shape[i]:
            raise ValueError('Shape of minibatch and variable are incompatible')
    
    indices = np.load(os.path.join('DATA', 'MINIBATCHES', f'{index}_indices.npy'))
    values = np.load(os.path.join('DATA', 'MINIBATCHES', f'{index}_values.npy'))
    sparse_tensor = tf.sparse.SparseTensor(indices=indices, values=values, dense_shape=shape)
    var.assign(tf.cast(tf.sparse.to_dense(sparse_tensor), dtype=tf.float32))

In [3]:
var = tf.Variable(tf.zeros([500, 30, 10002]), dtype=tf.float32, shape=[None, 30, 10002])

load_minibatch(0, var)

In [5]:
import os
import json

with open(os.path.join('DATA', 'numberize.json'), 'r') as f:
    numberize = json.load(f)
        
with open(os.path.join('DATA', 'reverse_numberize.json'), 'r') as f:
    reverse_numberize = json.load(f)

In [32]:
def verbalize_sentence(numberized_sentence):
    words = [reverse_numberize[str(word_number)] for word_number in numberized_sentence if word_number != 10001]
    return ' '.join(words)

def verbalize(vectorized_sentences):
    numberized_sentences = tf.argmax(vectorized_sentences, axis=2).numpy()
    return list(map(verbalize_sentence, numberized_sentences))

In [88]:
class BatchGenerator(object):

    def __init__(self, num_batches):
        self.num_batches = num_batches
        self.current_idx = 0
        self.x = tf.Variable(tf.zeros([500, 30, 10002]), dtype=tf.float32, shape=[None, 30, 10002])
        self.y = tf.Variable(tf.zeros([500, 30, 10002]), dtype=tf.float32, shape=[None, 30, 10002])
        
    def generate(self):
        while True:
            load_minibatch(self.current_idx, self.x)
            self.y[:, 0, :].assign(tf.zeros([500, 10002]))
            for i in range(0,29):
                self.y[:, i+1, :].assign(self.x[:, i, :])
            self.current_idx = (self.current_idx + 1) % self.num_batches
            yield tf.identity(self.x), tf.identity(self.y)
            
num_batches = 6807
batch_generator = BatchGenerator(6807).generate()

In [136]:
from keras import Sequential
from keras.layers import LSTM, TimeDistributed, Dense, Activation

model = Sequential()
model.add(LSTM(256, input_shape=(None, 10002), return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(10002)))
model.add(Activation('softmax'))

In [137]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [138]:
batches_to_process = 5

model.fit_generator(batch_generator, batches_to_process, 1)

Epoch 1/1
5/5 [==============================] - 623s 125s/step - loss: 8.8541 - categorical_accuracy: 0.3931


In [146]:
word_vec = np.zeros((1, 1, 10002))
word_vec[0, 0, 1] = 1

In [147]:
res = model.predict(word_vec)

In [148]:
np.argmax(res, axis=2)

array([[10001]])